### This code imports commodity prices from the IMF database.


#### Packages
- Smith, C. C. (2023). Package IMFP (1.1) [Python]. https://github.com/chriscarrollsmith/imfp
- Package Pandas (2.2). (2024). [Python]. https://pandas.pydata.org/


In [1]:
#import pandas as pd
import imfp
import useful_functions as uf

In [2]:
# Check the available databases
databases = imfp.imf_databases()

# Fetch list of valid parameters and input codes for commodity price database
params = imfp.imf_parameters("PCPS")

In [ ]:
# Fetch the 'freq' input code for Monthly frequency
selected_freq = list(
    params['freq']['input_code'][params['freq']['description'].str.contains("Monthly")]
)

# List of substrings to check for
commodities_codes = ["PCOFFOTM", #Coffee, Other Mild Arabica
                   "PIORECR", #Iron Ore
                   "PMEAT", #PMEAT index
                   "POILBRE", #Brent Crude Oil
                   "PORANG", #Orange
                   "PSOYB", #Soybeans
                   "PSUGA" #Sugar index
                   ]

# Fetch the 'commodity' input code for coal
selected_commodity = list(
    params['commodity']['input_code'][params['commodity']['input_code'].isin(commodities_codes)]
)

# Fetch the 'unit_measure' input code for index
selected_unit_measure = list(
    params['unit_measure']['input_code'][params['unit_measure']['description'].str.contains("Index")]
)

# Request data from the API
df_commo = imfp.imf_dataset(database_id = "PCPS",
         freq = selected_freq, commodity = selected_commodity,
         unit_measure = selected_unit_measure,
         start_year = 2001, end_year = uf.define_end_period("%Y")) # Fetch data from 2001 to the month before the current one

# Filter the columns
df_commo = df_commo[['time_period', 'commodity', 'obs_value']]
df_commo = df_commo.rename(columns={'time_period': 'Date', 'obs_value': 'value'}) # Rename columns
# Replace the input codes with the actual names of the commodities
df_commo['commodity'] = df_commo['commodity'].replace({
    'PCOFFOTM': 'WD_Coffee',
    'PIORECR': 'WD_Iron Ore',
    'PMEAT': 'WD_Meat index',
    'POILBRE': 'WD_Brent Crude Oil',
    'PORANG': 'WD_Orange',
    'PSOYB': 'WD_Soybeans',
    'PSUGA': 'WD_Sugar'
})

In [ ]:
# Pivot the data frame
df_commodities = df_commo.pivot(index='Date', columns='commodity', values='value')

# Save the data frame to a CSV file
df_commodities.to_csv('../data/df_commodities.csv')